# Female diabetes predictor

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
data=pd.read_csv("../data/diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
y=data.Outcome
x=data[["Pregnancies","BloodPressure","BMI","Age"]]
x_train, x_test, y_train, y_test=train_test_split(x,y)

In [5]:
param_grid = {'n_estimators': [1, 10, 100, 1000], 'max_depth': [None, 2,5,10]}

grid_diabestes=GridSearchCV(estimator=RandomForestClassifier(class_weight="balanced"), param_grid=param_grid, cv=5)
grid_diabestes.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 10, 100, 1000], 'max_depth': [None, 2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [6]:
diabetes_model=grid_diabestes.best_estimator_

In [7]:
y_pred=diabetes_model.predict(x_test)
y_pred_prob=diabetes_model.predict_proba(x_test)
#confusion_matrix(y_test,np.array(list(zip(*y_pred_prob)))[1]>.5)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[81 52]
 [ 9 50]]
             precision    recall  f1-score   support

          0       0.90      0.61      0.73       133
          1       0.49      0.85      0.62        59

avg / total       0.77      0.68      0.69       192



In [8]:
from sklearn_porter import Porter

porter = Porter(diabetes_model, language='js')
output = porter.export()


with open('../src/js/diabetes_model.js' , 'w') as f:
    f.write(output)